<a href="https://colab.research.google.com/github/ZeeeeCS/MarketCanadaStock/blob/main/Marketanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
import plotly.io as pio
pio.templates.default = "plotly_white"

In [ ]:

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download "atharvaarya25/financials"

rm: cannot remove '/root/.kaggle': No such file or directory
mv: cannot stat './kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/atharvaarya25/financials
License(s): DbCL-1.0
  0% 0.00/21.5k [00:00<?, ?B/s]
100% 21.5k/21.5k [00:00<00:00, 36.2MB/s]


In [ ]:

! unzip /content/financials.zip

Archive:  /content/financials.zip
  inflating: Financials.csv          


In [ ]:
data=pd.read_csv("/content/Financials.csv")

In [ ]:
data=data[data['Segment']=='Government']

In [ ]:
data=data[data['Country']=='Canada']



In [ ]:
data.columns

Index(['Segment', 'Country', ' Product ', ' Discount Band ', ' Units Sold ',
       ' Manufacturing Price ', ' Sale Price ', ' Gross Sales ', ' Discounts ',
       '  Sales ', ' COGS ', ' Profit ', 'Date', 'Month Number',
       ' Month Name ', 'Year'],
      dtype='object')

In [ ]:
data[' Product '].unique()

array([' Carretera ', ' Paseo ', ' VTT ', ' Montana ', ' Velo ',
       ' Amarilla '], dtype=object)

In [ ]:
data['  Sales ']=data['  Sales '].str.replace('$','').str.replace(',','').astype(float)

In [ ]:
data[' Profit ']=data[' Profit '].str.replace('$','').str.replace(',','').astype(float)

In [ ]:
sales_by_product=data.groupby(' Product ')['  Sales '].sum()

In [ ]:
sales_by_product.reset_index()

,Product,Sales
0,Amarilla,2579791.79
1,Carretera,1332156.65
2,Montana,829859.91
3,Paseo,3956873.45
4,VTT,667982.25
5,Velo,1374572.47


In [ ]:
Profit=data.groupby('Year')[' Profit '].sum()

In [ ]:
Profit

,Profit
Year,
2013,606726.12
2014,1651745.40


In [ ]:
data[' Gross Sales ']=data[' Gross Sales '].str.replace('$','').str.replace(',','').astype(float)

In [ ]:
gross_profit=data.groupby('Year')[' Gross Sales '].sum()

In [ ]:
gross_profit

,Gross Sales
Year,
2013,2589198.0
2014,9056159.0


In [ ]:
data[' Discounts ']=data[' Discounts '].str.replace('$','').str.replace(',','').str.replace('-','').str.replace('    ','0').astype(float)

In [ ]:
services=data.groupby('Year')[' Discounts '].sum()
services

,Discounts
Year,
2013,110771.88
2014,793348.60


In [ ]:
data[' COGS ']=data[' COGS '].str.replace('$','').str.replace(',','').astype(float)

In [ ]:
cost_of_revenue=data.groupby('Year')[' COGS '].sum()

In [ ]:
product_cost=cost_of_revenue.subtract(services)



In [ ]:
services

,Discounts
Year,
2013,110771.88
2014,793348.60


In [ ]:
revenue=gross_profit + product_cost

In [ ]:
r_and_d = 1200
sg_a = 1500
taxes = 1800
operating_expences=(r_and_d + sg_a)
operating_profit = gross_profit - (r_and_d + sg_a)
net_profit = operating_profit - taxes

In [ ]:
labels = [
    "Carretera", "Paseo", "VTT", "Montana", "Velo", "Amarilla","Products",
    "Revenue", "Cost of revenue","services cost","Products Cost", "Gross Profit","Operateing Expenses","R&D", "SG&A", "Taxes","Operating Profit", "Net Profit"
]

In [ ]:
source = [0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 8, 7, 11, 12, 12, 16, 11, 16]
target = [6, 6, 6, 6, 6, 6, 7, None, 8, 9, 10, 11, 12, 12, 12, 12, 16, None]
value = [sales_by_product[0],sales_by_product[1],sales_by_product[2],sales_by_product[3],sales_by_product[4],sales_by_product[5],revenue[2014],revenue[2014],cost_of_revenue[2014],services[2014],product_cost[2014],gross_profit[2014],operating_expences,r_and_d,sg_a,taxes,operating_profit,net_profit[2014]]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black"),
        label=labels # Add labels to nodes
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
))
fig.update_layout(title_text="Canada Stock", font_size=10)
fig.show()
